<h1>PredictCrystalline.ipynb </h1>

It takes the files from CrystallineFileLecturePredict.ipynb and predicts with them. There is now only one function that can be used for prediction, Predict.
**IMPORTANT, IN THE FOLDER CrystallineModels YOU HAVE TO MANUALLY PASTE THE FOLDERS WITH THE MODELS. IT IS A MANUAL PROCESS. THE REST OF THE PROCESS SHOULD BE AUTOMATED EXCEPT FOR THE TIME CONDITIONS THAT ARE ASKED TO THE USER**

Here is a diagram of the folder output structure:
        A folder structure with the predictions:
        
        - PredictCrystalline.ipynb
        - CrystallinePredictionsFolder
          |
          |_Model_{NameOfModel 1}
          |    |
          |    |_Experiment_{NameOfExperiment 1}
          |    |   |
          |    |   |_Graph with predictions and original data (.png)
          |    |   |
          |    |   |_Text file with the results. First column is the internal program time
          |    |   |                             Second column is the time in real life of those predictions (YY-MM-DD HH:mm:SS
          |    |   |                             Third column is the predicted polarization
          |    |   |                             Fourth column is the uncertainty
          |    |   |
          |    |   |_PolarizationD3_{Name}_Paramteres.txt (original parameter file)
          |    |   |_PolarizationD3_{Name}.txt (original data file)
          |    |   
          |    |_Experiment_{NameOfExperiment 2}
          |    |   |
          |    |   |_Graph with predictions and original data (.png)
          |    |   |
          |    |   |_Text file with the results.  
          |    |   |
          |    |   |_Parameter and array original files
          |    |
          |    |_(...)
          |
          |_Model_{NameOfModel 2}
          |    |
          |    |_Experiment_{NameOfExperiment 1}
          |    |   |
          |    |   |_Graph with predictions and original data (.png)
          |    |   |_Text file with the results. First column is the internal program time
          |    |   
          |    |_Experiment_{NameOfExperiment 2}
          |    |   |
          |    |   |_Graph with predictions and original data (.png)
          |    |   |_Text file with the results.   
          |    |   |_Parameter and array original files
          |    |
          |    |_(...)
          |
          |_Combined model
          |    |
          |    |_Experiment_{NameOfExperiment 1}
          |    |   |
          |    |   |_Graph with predictions and original data (.png)
          |    |   |_Text file with the results. First column is the internal program time
          |    |   
          |    |_Experiment_{NameOfExperiment 2}
          |    |   |
          |    |   |_Graph with predictions and original data (.png)
          |    |   |
          |    |   |_Text file with the results.  
          |    |   |
          |    |   |_Parameter and array original files   
          |    |
          |    |_(...)
          |_(...)



The inputs of the code are:
1. The point in the past to start predicting (e.g. if you don't want to start your predictions at the time of the first measurement you can write -123 and the predictions will start 123 seconds before the first experimental measurement). Accepts positive and negative values and I believe even floats (not tested for non-integer values but I don't think it makes sense to ask for time values with a precision smaller than what the files from D3 have)
2. The time between steps (the time interval between predictions, e.g, if you want predictions every 4 seconds, write as the second input 4).
3. The point in the future to stop predicting (e.g. if you don't want to stop at the last experimental measurement you can write 1024 and the last polarization will be the last step before t_final+1024). Accepts positive and negative values




___________________________________________________________________________________________

 
OUTPUTS OF THE CODE (the important ones)

1. **CrystallineLogFile_PredictingML.txt**
It is a log with all the important steps that the code has done

2. **AbsoluteTimes.txt**
It is only created if you use Predict (not with PredictWithPolarizationTimeReference). It stores for each experiment (and model, if I have time I will remove those duplications) the name of the experiment, the time where the first polarization measurement was recorded (in an absolute format like "2023-12-08 13:06:39") and the time value that was used for the Predict function (it is just DiffractogramAbsoluteTime that was added so there is a txt file with all the information needed to change from time intervals with random reference points to absolute time strings if needed)

3. **CrystallineExecution_times.txt**
It records the time it took each individual experiment to be fully predicted. The time depends on the amount of points and model but 48 full predictions with 20 second gaps took less than 90 seconds (hopefully it is fast enough) 

4. **CrystallinePredictionsFolder**
Stores all the predictions. The first subdivision of folders separate the information using the model

    4.1 **Model\_{Complexity}\_{num\_augmentations}**
Depending of the model (characterized using two strings) you can obtain predictions for all the experiments. In each Model_{Complexity}_{num_augmentations} folder you will find folders for each experiment

        4.1.1 Experiment_{base_name}
For each model and {base_name} (or experiment name) you can find one of these folders. The contents contain the predictions

            4.1.1.1 {Complexity}_{num_augmentations}_{base_name}.jpg
Contains the pure ML predictions in red and the correction (in green) using the first and final polarization measurements. For more information read the explanation of what the code does.
 
            4.1.1.2 PredictedData_{Complexity}_{num_augmentations}_{base_name}
Contains the same information as the green curve but on a txt file. On the first column you have the relative time where t=0 corresponds to the first correct polarization measurement. On the second column you have the absolute time (in a format datetime.strptime("2023-12-08 13:06:39", "%Y-%m-%d %H:%M:%S") ) and on the second and third columns you can find the polarization and uncertainty of polarization associated to those time values



___________________________________________________________________________________________


Information about the parts



Here we have functions that train, validate and fit the models. Some models require the variables to be scaled or will scale them. Extra precautions need to be taken into account
1. _PrintDebug_ is a flag that allows the code to output on screen all the steps. If it is set to false, it won´t show anything. However, all information will be properly logged whether this flag is set to true or false. The name of the log is determined by the variable *log_file_path*. The code runs faster if it is set to False.

2. _ShowPlot_ is a similar flag that allows the code to show on screen all plots that are being produced. They are all stored independently of whether this flag is True or False. The code runs faster if it is set to False.

3. _LogNoise_ is another flag that allows numerical values in the log. Most of these values are not worth keeping but if you want to see if there are no NaNs or zeros you can turn it on


4. **log_message** is a function used for writting on the log file

5. **win_long_path** is a function that "fixes" directory paths

6. **load\_experiments**. It uses the directory where the array (numerical values) and parameter files resides, picks one polarization column and prepares a list of experiments to feed the ML algorythm. The output is as follows:

    *encoded\_experiments=[(static\_values, Deltatime, PolarizationD3, ErrPolarizationD3)$_{experiment 1}$,(static\_values, Deltatime, PolarizationD3, ErrPolarizationD3)$_{experiment 2}$,...]* 
    
    Note that the Cell Id is Hot Encoded. The type of cell did not affect greatly the predictions. However in the future, it may be useful to give more information to Cell_ID. As of 2025, these strings are Hot Encoded which means that the code finds all the different types, creates columns for each type and writes 0 or 1 (bool) depending of whether the cell was from one type or the other. This is the standard procedure to feed categorical variables to ML.
    The size of the output list depends on the number of pairs of .txt files present in the folder. This means that it also works for isolated experiments.
    
7. **build_dataset** is a function that prepares the data base to be fed directly into a ML model for training and validation. There are a few import decisions taken here. The way this function is set up, it removes 2 rows of data per polariser cell studied. The reason why it was done is because we want the ML model to be able to predict polarization decay when given the specs of the cell (the static parameters) and the initial and final polarization values (with their associated time values). The reason why those two values are considered "known values" for each cell is beacuse they can be easily measured experimentally and they give as a good estimate about the overall behaviour. In some experiments, the environment of the studied sample is too fragile to move and place the Si crystal for polarization measurements. Therefore, a working ML algorithm whose inputs are the specs of the cell and the initial and final polarizations (measured before and after the sample is in place) would enable experiments that could not be done before without proper polarization efficiency corrections. 
Also, we remove those two values per polariser cell from the training arrays (Xt, y and err). The reason why it is done is to avoid data leaks in the model. If we give those values as training data and also give them as parameters, the ML algorithm can run into the risk of memorizing those pairs (over-fitting) and worsen any new predicitions. Uncertainties for the first and last polarization measurements are not added to the static features. This was a decision taken to avoid giving too much weight to two variables that don't have value on their own (they compliment the polarization values but if the ML architecture is as shallow as
the one used here, they can be considered independent variables and reduce the weight and importance of the other variables). 
Another consideration taken here was that the static features get duplicated in Xs a lot of times. One could think that using a similar method that the one used for augmentations of the data sets could also diversify the data base. However, we wanted all measurements of a same session and cell to be coherent 
and it wouldn't make sense to have different static features. Therefore, the safest approach was to only duplicate these values. For the augmented experiments the only parameters that are changed are the initial and final times and polarizations. There is no incompatibility here to what we have just said as these work as "hypothetical independent experiments". This is why augmentation is done before this function gets used.
         

8. **nll_loss** ML algorythms require a way to tell the algorythm if it is learning or not. The most standard practice is with a Loss function. If the loss value goes down that means that the algorythm is learning and, if a step increases the loss, then it is punished and tries other approaches. When using uncertainties when teaching the model, the most common loss function is the NLL or Negative log-likelihood of a normal distribution 
NLL$=\frac{1}{2}$log$(σ^2)+\frac{(y−μ)^2}{2σ^2}$
where $\sigma$ is the uncertainty in the predictions, $y$ is the measured value and $\mu$ the predicted value. Instead of predicting $σ^2$ directly, we obtain its logarithm to have a more stable process (and avoid accounting precision as $\sigma^{-2}$ which is numerically unstable when uncertainties are low).

However we want to avoid uncertainties that drift too far from the overall model predictions. To achieve that, we can get a rough estimate on what the uncertainty of a set predictions looks like.
Let $\vec{\mu}=\left(\mu_1,\ldots,\mu_N\right)^T$ be the vector of $N$ predicted values. It can be considered as a random vector of variance:
$Var(\vec{\mu})=\frac{1}{N}\sum_{i=1}^N\left(\mu_i-E(\vec{\mu})\right)$
where $E(\vec{\mu})$ is the mean of the predicted values. We then have two different variances, one obtained as the sparseness of the predictions, (denoted as $Var\left(\vec{\mu}\right)$, and one obtained as a result of the internal ML calculations (denoted as $\sigma^2$). A penalty can be added to the loss functions to force the model to try to reduce this differences. An easy way to model it is to obtain the difference of those variances and square the result (taking the absolute value was also a good estimate, but using squared values punished big discrepancies in a stronger way).

$StrayPenalty = B \cdot \left[\log\left(\sigma^2\right)-\log\left(\mathrm{Var}\left(\vec{\mu}\right)\right)\right]^2$
where $B$ is a constant used to control the weight of this penalty. The reason why $Var\left(\vec{\mu}\right)$ was used and not $Var\left(\vec{y}\right)$ (with $\vec{y}$ the vector of measured values) was to avoid noise in the original data to tamper with the loss function. It would be physically clearer to use measured values sparseness as a way to guide the model but some experimental uncertainties are clearly underestimated and that would cause this penalty to dominate the loss and obscure the main loss protocol, the NLL.

Also, we also want to punish the model if it tries overestimating $\sigma$. If the model is unable to minimize $y-\mu$, in order to lower NLL, it increases $\sigma$. If no precautions are taken, this "escape solution" achieves bad predictions with inflated uncertainties that simulate a low loss value. A new penalty was added that punishes overestimation of the uncertainties more than underestimation (which never happened). The slope correction done further on the pipeline can "fix" this issue but what the model returns then is closer to a poorly calculated linear fit
Therefore, an addition penalty was added.

$OverestimatePenalty= C \cdot \max\left(0, \log\left(\sigma^2\right)-\log\left(\mathrm{Var}(\vec{\mu})\right)\right)$
where $C$ is a constant used to control the weight of this penalty

9. **model\_fitting**. It is a function that logs and runs model.fit() on a two-input Keras model and returns the training history. It needs **static, time and polarization variables scaled**. Training is not done using the uncertainties of the data as it was decided that uncertainty information is encoded in the augmentations. Note: No validation is done anywhere in the code. Here are some of the reasons:
    
    9.1. The data base is very small. The amorphous data base contains only 199 points while the crystalline one contains 251. Removing a small percentage of those points for validation might leave the data base too small and underfitting might worsen the result more than fine tuning parametrs with validation.
    
    9.2. A randomized validation split may be physically wrong. Therefore it should be chronological, not shuffled. However, in crystalline experiments, there are decay experiments that have only four or five intermediate points. Even removing one point for validation is a massive hit on the experiment. Therefore, it is risky to add validation
    
    9.3. To find good models, a Leave-one-out approach was used. For a certain model structure, an experiment gets removed and the model and it trains on all the remaining experiments. Then, the model tries to predict this isolated experiment. Afterwards, the experiment is returned and a new one becomes isolated. This process loops for all experiments and an overall score of the model is computed. This process was done for 498 models for crystalline materials. This is a stronger (and more expensive) method than validation as it is not dependent on the validation splits and avoids possible information leaks.

Also, eight randomly picked models were tested with and without validation and with and without an asymetric uncertainty-overestimated penalizing loss. The result showed that the Loss update was an improvement and validation did not increase performance (without validation, the results were slightly better).

10. **model\_prediction**. It is a funtion that predicts with a given model. It needs **static and time variables scaled**. This scaling must be coherent to the one done in the rest of the funtions.

111. **train**. This function is the one responsible of scaling the inputs and training the model (it uses **model\_fitting**)

     11.1. It creates the independent arrays with all the encoded experiments (augmented or not) using build_dataset
    
     11.2. Then it scales the data. ML algorithms work better when the inputs and outputs are normalized. The reason why we don't normalize inside the function is to have those scaler defined globally and not locally
    
     11.3. It builds the model depending on the use\_uncertainty bool. (It changes the loss function and the output).
    
     11.4. It trains the model and returns its history (the trained model)
    
12. **align_static_vectors**. It converts the columns not present on an isolated experiment to zeros.
    
13. **model_predict_sloped** It substracts a linear function to the predicted values. If done correctly this makes it so that the polarization predictions at the initial and final time points are the same as the measured polarization values at those times. This fixes a vertical shift and also an overall slope. As it is a correction done with experimental values, the algorithm is still "universal". However we can't fully say that it is a pure ML algorithm. The "correctness" of this method is subjective. It is a warning in the ML front that there is an issue with the data base but it is a valid fix for experimentalists.
___________________________________________________________________________________________

Process it follows:

The steps it takes are explained here:
1. Obtain the absolute time references for the experiments (year, month, day, hour, minute and second)
2. Loop over all models that will be used
3. Loop over all two-numerical-rowed experiments. It copies them (does't move them) to the final folders
4. Build the structure that the ML model was trained with and scale all the experiment using the scalers that the model used in training
5. Predict, correct the slope if Correction=True, plot everything and write the results on a .txt file

Finally, it runs the _Predict_ function. The code will not progress until the user manually inputs the time values. It asks for three values, t_initial (which means that predictions start _t_initial_ seconds later (it can be negative)), _t_final_ (it changes the final time point for predictions by t_final seconds) and _t_step_ (the time steps between predictions)
It also takes all previous predictions of all models and averages them using a weighted mean. That way we have a single prediction file that should be the best that ML can currently do.
For every experiment, the code finds all the experiment files created by each model. Then, for every line it performs the following calculation. Let $P_1\left(t\right),\ldots,P_n\left(t\right)$ be the polarization predictions of models $1,\ldots,n$ at time $t$ with uncertainties $\sigma_1\left(t\right),\ldots,\sigma_n\left(t\right)$. Then the weighted mean is 

$P\left(t\right)=\frac{\sum_{i=1}^n P_i\left(t\right)\sigma_i^{-2}\left(t\right)}{\sum_{i=1}^n \sigma_i^{-2}\left(t\right)}$

with

$\sigma^2\left(t\right)=\frac{1}{\sum_{i=1}^n \sigma_i^{-2}\left(t\right)}$

## 1. Libraries

In [ ]:
import os
import glob
import math
import shutil
import gc
import joblib
import sys
from datetime import datetime, timedelta
import time as pytime
import pandas as pd
import numpy as np
import tensorflow as tf
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras import Input, Model, regularizers
from tensorflow.keras.layers import Dense, Concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import backend as K

## 2. Auxiliary Functions and log file creation

1. _PrintDebug_ is a flag that allows the code to output on screen all the steps. If it is set to false, it won´t show anything. However, all information will be properly logged whether this flag is set to true or false. The name of the log is determined by the variable *log_file_path*. The code runs faster if it is set to False.

2. _ShowPlot_ is a similar flag that allows the code to show on screen all plots that are being produced. They are all stored independently of whether this flag is True or False. The code runs faster if it is set to False.

3. _LogNoise_ is another flag that allows numerical values in the log. Most of these values are not worth keeping but if you want to see if there are no NaNs or zeros you can turn it on


3. **log_message** is a function used for writting on the log file

4. **win_long_path** is a function that "fixes" directory paths

In [ ]:
PrintDebug = False #This Bool will determine if all logs should be printed on screen on the Python Notebook. The log writing is always on. If False the code will be faster.
ShowPlot = False #This Bool works the same but with showing on screen the plots (they are always saved even with this variable being False). Reduces program cost if False
LogNoise = False #This Bool allows numerical values in the log. Most of these values are not worth keeping but if you want to see if there are no NaNs or zeros you can turn it on
log_file_path = "CrystallineLogFile_PredictingML.txt"
def log_message(message):
    """
    Arguments: 
        message (string): The text that will be logged
    
    Returns:
        None
        
    Notes:
        It will write the string "message" in the log file.
        If PrintDebug==True then it will also print the string
    """
    message = str(message)
    if PrintDebug:
        print(message)
    with open(log_file_path, 'a', encoding='utf-8') as log_file:
        log_file.write(str(message) + "\n")
        
################################################################

def long_path(path):
    """
    Arguments: 
        path (path): The path that needs to be converted
    
    Returns:
        The updated path string or path depending on the platform used
        
    Notes:
        To avoid Windows 260 character limit for Windows paths, a special "prefix" is added.
        It also unifies how directories are managed.
        Also works with Linux and Mac

    """
    # Convert to Path and resolve to absolute
    path = Path(path).resolve()
    
    #Windows only:  
    if os.name == "nt":
        path_str = str(path)
        if not path_str.startswith("\\\\?\\"):
            # UNC paths need special handling
            if path_str.startswith("\\\\"):
                path_str = "\\\\?\\UNC\\" + path_str[2:]
            else:
                path_str = "\\\\?\\" + path_str
            return path_str
    
    return path

to_erase = [
    "CrystallineLogFile_PredictingML.txt",
    "CrystallineExecution_times.txt",
    "CrystallinePredictionsFolder"]

for item in to_erase:
    path = os.path.abspath(item)
    if os.path.exists(path):
        try:
            if os.path.isfile(path):
                os.remove(path)
                log_message(f"Deleted file: {path}")
            elif os.path.isdir(path):
                shutil.rmtree(path)
                log_message(f"Deleted folder: {path}")
        except Exception as e:
            log_message(f" Could not delete {path}: {e}")
    else:
        log_message(f"Not found (skipped): {path}")
        
with open(log_file_path, 'w', encoding='utf-8') as log_file:
    log_file.write("=== Log started ===\n")
    log_file.write("All outputs from functions have a string at the beginning to show the origin:\n")

## 3. Functions

1. **load\_experiments**. It uses the directory where the array (numerical values) and parameter files resides, picks one polarization column and prepares a list of experiments to feed the ML algorythm. The output is as follows:

    *encoded\_experiments=[(static\_values, Deltatime, PolarizationD3, ErrPolarizationD3)$_{experiment 1}$,(static\_values, Deltatime, PolarizationD3, ErrPolarizationD3)$_{experiment 2}$,...]* 
    
    Note that the Cell Id is Hot Encoded. The type of cell did not affect greatly the predictions. However in the future, it may be useful to give more information to Cell_ID. As of 2025, these strings are Hot Encoded which means that the code finds all the different types, creates columns for each type and writes 0 or 1 (bool) depending of whether the cell was from one type or the other. This is the standard procedure to feed categorical variables to ML.
    The size of the output list depends on the number of pairs of .txt files present in the folder. This means that it also works for isolated experiments.
    
3. **build_dataset** is a function that prepares the data base to be fed directly into a ML model for training and validation. There are a few import decisions taken here. The way this function is set up, it removes 2 rows of data per polariser cell studied. The reason why it was done is because we want the ML model to be able to predict polarization decay when given the specs of the cell (the static parameters) and the initial and final polarization values (with their associated time values). The reason why those two values are considered "known values" for each cell is beacuse they can be easily measured experimentally and they give as a good estimate about the overall behaviour. In some experiments, the environment of the studied sample is too fragile to move and place the Si crystal for polarization measurements. Therefore, a working ML algorithm whose inputs are the specs of the cell and the initial and final polarizations (measured before and after the sample is in place) would enable experiments that could not be done before without proper polarization efficiency corrections. 
Also, we remove those two values per polariser cell from the training arrays (Xt, y and err). The reason why it is done is to avoid data leaks in the model. If we give those values as training data and also give them as parameters, the ML algorithm can run into the risk of memorizing those pairs (over-fitting) and worsen any new predicitions. Uncertainties for the first and last polarization measurements are not added to the static features. This was a decision taken to avoid giving too much weight to two variables that don't have value on their own (they compliment the polarization values but if the ML architecture is as shallow as
the one used here, they can be considered independent variables and reduce the weight and importance of the other variables). 
Another consideration taken here was that the static features get duplicated in Xs a lot of times. One could think that using a similar method that the one used for augmentations of the data sets could also diversify the data base. However, we wanted all measurements of a same session and cell to be coherent 
and it wouldn't make sense to have different static features. Therefore, the safest approach was to only duplicate these values. For the augmented experiments the only parameters that are changed are the initial and final times and polarizations. There is no incompatibility here to what we have just said as these work as "hypothetical independent experiments". This is why augmentation is done before this function gets used.
         

4. **nll_loss** ML algorythms require a way to tell the algorythm if it is learning or not. The most standard practice is with a Loss function. If the loss value goes down that means that the algorythm is learning and, if a step increases the loss, then it is punished and tries other approaches. When using uncertainties when teaching the model, the most common loss function is the NLL or Negative log-likelihood of a normal distribution 
NLL$=\frac{1}{2}$log$(σ^2)+\frac{(y−μ)^2}{2σ^2}$
where $\sigma$ is the uncertainty in the predictions, $y$ is the measured value and $\mu$ the predicted value. Instead of predicting $σ^2$ directly, we obtain its logarithm to have a more stable process (and avoid accounting precision as $\sigma^{-2}$ which is numerically unstable when uncertainties are low).

However we want to avoid uncertainties that drift too far from the overall model predictions. To achieve that, we can get a rough estimate on what the uncertainty of a set predictions looks like.
Let $\vec{\mu}=\left(\mu_1,\ldots,\mu_N\right)^T$ be the vector of $N$ predicted values. It can be considered as a random vector of variance:
$Var(\vec{\mu})=\frac{1}{N}\sum_{i=1}^N\left(\mu_i-E(\vec{\mu})\right)$
where $E(\vec{\mu})$ is the mean of the predicted values. We then have two different variances, one obtained as the sparseness of the predictions, (denoted as $Var\left(\vec{\mu}\right)$, and one obtained as a result of the internal ML calculations (denoted as $\sigma^2$). A penalty can be added to the loss functions to force the model to try to reduce this differences. An easy way to model it is to obtain the difference of those variances and square the result (taking the absolute value was also a good estimate, but using squared values punished big discrepancies in a stronger way).

$StrayPenalty = B \cdot \left[\log\left(\sigma^2\right)-\log\left(\mathrm{Var}\left(\vec{\mu}\right)\right)\right]^2$
where $B$ is a constant used to control the weight of this penalty. The reason why $Var\left(\vec{\mu}\right)$ was used and not $Var\left(\vec{y}\right)$ (with $\vec{y}$ the vector of measured values) was to avoid noise in the original data to tamper with the loss function. It would be physically clearer to use measured values sparseness as a way to guide the model but some experimental uncertainties are clearly underestimated and that would cause this penalty to dominate the loss and obscure the main loss protocol, the NLL.

Also, we also want to punish the model if it tries overestimating $\sigma$. If the model is unable to minimize $y-\mu$, in order to lower NLL, it increases $\sigma$. If no precautions are taken, this "escape solution" achieves bad predictions with inflated uncertainties that simulate a low loss value. A new penalty was added that punishes overestimation of the uncertainties more than underestimation (which never happened). The slope correction done further on the pipeline can "fix" this issue but what the model returns then is closer to a poorly calculated linear fit
Therefore, an addition penalty was added.

$OverestimatePenalty= C \cdot \max\left(0, \log\left(\sigma^2\right)-\log\left(\mathrm{Var}(\vec{\mu})\right)\right)$
where $C$ is a constant used to control the weight of this penalty

In [ ]:
def load_experiments(data_dir, polarization_column):
    """
    Arguments:
        data_dir (str): The direction to the folder where all files to be loaded will be found
        polarization_column: It is the name in the header (in the .txt files) of the column that will be read.
                             It can be 'SoftPolarizationD3' or 'PolarizationD3'. All results were obtained with 'PolarizationD3'
    Output: 
        A list, for all polarization decays like:
        encoded_experiments = [
            (static_values,   # list of static parameters (per experiment)
            Deltatime,       # 1D numpy array of time values
            polarization,    # 1D numpy array of polarization values
            Uncertainty      # 1D numpy array of uncertainty values), ...]
        A pd object with the Hot encoded Ids and the rest of the parameters per experiment (each experiment in a different row)
    
    Notes:
    The steps the code does are the following:
    1. Finds all array files (the ones with the numerical values of the decay) and loops over all of them
    2. For each array files it reconstructs the name of the parameter file. It concatenates all parameter files into one pd structure.
    3. Finally it loops over all array files appending the parameters, time, polarization and uncertainty of every experiment to a common list
    """
    
    log_message(f"    load_experiments: Finding all Array Files...")
    # Step 1:

    arrays_files = sorted(
        glob.glob(os.path.join(data_dir, "*.txt"))) #Find all files that are .txt
    arrays_files = [f for f in arrays_files if not f.endswith("_Parameters.txt")] #Keep only the Arrays (not the parameters)

    encoded_experiments = []
    all_static_df = []
    static_columns = ['CellID', 'Pressure', 'LabPolarization', 'LabTime'] #Parameter header
    for arrays_path in arrays_files:
        base = os.path.basename(arrays_path)
        # Build parameters filename by adding _Parameters before .txt
        name_without_ext = os.path.splitext(base)[0]
        parameters_filename = f"{name_without_ext}_Parameters.txt"
        parameters_path = os.path.join(data_dir, parameters_filename)

        # Read parameters file
        try:
            parameters_df = pd.read_csv(parameters_path) #Import the parameter file
            if LogNoise:
                log_message(f"    load_experiments: Reading parameters file: {parameters_filename}") #Clutter logging

            #Get the first row as static data
            static_row = parameters_df.iloc[0][static_columns] #Get the parameter numerical values
            all_static_df.append(static_row)

        except Exception as e:
            log_message(f"    ****load_experiments: Failed to read parameters file: {parameters_filename}, error: {e}")
            continue

    log_message(f"    load_experiments: Create combined DataFrame for static parameters...")
    static_df = pd.DataFrame(all_static_df) #Combine all static rows into a Dataframe

    log_message(f"    load_experiments: Collected static data:")
    """
    The type of cell did not affect greatly the predictions. However in the future, it may be useful to give more information to the Cell_IDs.
    As of 2025, these strings are Hot Encoded which means that the code finds all the different types, creates columns for each type and writes
    0 or 1 (bool) depending of whether the cell was from one type or the other. This is the standard procedure to feed categorical varaibles to ML.
    """
    
    log_message(f"    load_experiments: Static dataframe columns:, {static_df.columns.tolist()}")
    log_message(f"    load_experiments: Static dataframe shape:, {static_df.shape}")

    log_message(f"    load_experiments: Hot encoding CellID.")
    categorical_cols = ['CellID']
    static_df = pd.get_dummies(static_df, columns=categorical_cols, prefix=['CellID'])
    # Now second pass: read arrays and create encoded_experiments with encoded static params
    for i, arrays_path in enumerate(arrays_files):
        base = os.path.basename(arrays_path)
        name_without_ext = os.path.splitext(base)[0]
        parameters_filename = f"{name_without_ext}_Parameters.txt"
        parameters_path = os.path.join(data_dir, parameters_filename)
        # log_message(f"Reading arrays file: {base}") #Clutter log
        arrays_df = pd.read_csv(arrays_path) # Reads the time series data 
    
        static_values = static_df.iloc[i].to_list() #Fetches the static parameters corresponding to this experiment
    
        Deltatime = arrays_df["DeltaTime"].values
        polarization = arrays_df[polarization_column].values #Extracts the time array and the selected polarization column.
        #Save the uncertainty even if it is not used afterwards
        Uncertainty = arrays_df["ErrPolarizationD3"].values
        #if len(Deltatime) > 2:
        encoded_experiments.append((static_values, Deltatime, polarization, Uncertainty))
        # log_message(f"Creating Encoded Experiments (appending parameters, time array, polarization array and uncertainty array)") #Clutter log
    log_message(f"    load_experiments: Loaded {len(encoded_experiments)} experiments.")
    return encoded_experiments, static_df.columns.tolist()


#######################################################################################3

def build_dataset(experiments, mode="PolarizationD3"):
    """
    Arguments:
        experiments (list): It is a list like the output of load_experiments or augment_experiments
            experiments = [
                (static_values,   # list of static parameters (per experiment)
                Deltatime,       # 1D numpy array of time values
                polarization,    # 1D numpy array of polarization values
                Uncertainty      # 1D numpy array of uncertainty values), ...]
        mode (str): The column of polarization that will be used. It can be either 'SoftPolarizationD3' or 'PolarizationD3'.
    
    Output:
        1. Xs. An array of lists. Shape (number_of_samples, number_static_features) Each list contains all the original static parameters (hot encoded) plus the first and last polarization values with uncertainty.
           To be precise, they get added in this order: static parameters + initial time + initial polarization + final time + final polarization.
           For all samples, the static features get added to this list. This means that, if an experiment has M measurements, two get added to the parameter
           list and then those parameter features get written M-2 times in this array
        2. Xt. An array of time. Shape (number_of_samples, 1). All time values that are not used as parameters get added to this array. However they are not
           saved directly. If they are for example 0,120,250 then they get saved as [0],[120],[250]. The reason is compatibility with Keras/TensorFlow.
        3. y. An array of polarization. It is the same as Xt but with polarization values (the type of polarization is determined by _mode_)
        4. err. An array of polarization uncertainties. It is the same as Xt and y but with polarization uncertainties.
    Notes:
        If there are only two rows then the file gets skipped. It shouldn't happen but there is logic for it. The reason why it gets skipped is because
        there would not be any values left to use for training or validation    
    """

    Xs, Xt, y, u = [], [], [], []
    log_message(f"    build_dataset: Starting build_dataset for column {mode} ")
    log_message(f"    build_dataset: Number of experiments to process: {len(experiments)} (should be (num_augmentations+1)*(number_of_experiments-1)")
    
    for exp_idx, (static_params, delta_time, polarization, uncertainty) in enumerate(experiments):
        if len(delta_time) < 2:
            log_message(f"    ****build_dataset:       Skipping experiment {exp_idx}: too few data points (len={len(delta_time)})")
            continue
            

        
        if LogNoise:
            log_message(f"    build_dataset: Adding First and Last Polarization (with time) values as static parameters") #Clutter log
        init_idx = 0
        final_idx = -1
        initial_dt, initial_p = delta_time[init_idx], polarization[init_idx]
        final_dt, final_p = delta_time[final_idx], polarization[final_idx]

        static_vector = static_params + [
            initial_dt, initial_p,
            final_dt, final_p
        ]
        if LogNoise:
            log_message(f"    build_dataset: Experiment {exp_idx}: static_vector length={len(static_vector)} (should be 10 (three parameters, CellID hot encoded creates three posibilities, four for the initial and final polarization) ") #Clutter log
        if LogNoise:
            log_message(f"    build_dataset: Building samples Static+time+polarization") #Clutter log
        
        for t, p, err in zip(delta_time[1:-1], polarization[1:-1], uncertainty[1:-1]): 
            Xs.append(static_vector)
            Xt.append([t])
            y.append(p)
            u.append(err) #We will ignore always uncertainty in parameters and even if they are not used, we will keep uncertainties in the data sets(same dimensions everywhere)

    log_message(f"    build_dataset: Number of experiments processed for mode '{mode}': {len(experiments)}")
    log_message(f"    build_dataset: Final dataset shapes: Xs: {np.array(Xs).shape}, Xt: {np.array(Xt).shape}, y: {np.array(y).reshape(-1, 1).shape}")
    return np.array(Xs), np.array(Xt), np.array(y).reshape(-1, 1), np.array(u).reshape(-1, 1)


#################################################################################

def split_experiments(experiments, val_fraction=0.2, seed=42):
    # Tests of validation. Currently unused
    rng = np.random.default_rng(seed)
    n_exp = len(experiments)
    indices = rng.permutation(n_exp)

    n_val = int(val_fraction * n_exp)
    val_idx = indices[:n_val]
    train_idx = indices[n_val:]

    train_experiments = [experiments[i] for i in train_idx]
    val_experiments   = [experiments[i] for i in val_idx]

    log_message(f"Split experiments: {len(train_experiments)} train / {len(val_experiments)} val")
    return train_experiments, val_experiments

#################################################################################

def nll_loss(y_true, y_pred):
    """
    Arguments:
        y_true (array): An array of the real values.
        y_pred (array): An array of the predicted values
    Output:
     A scalar tensorflow tensor ( () ) with the value of the loss as the the mean Gaussian negative log-likelihood over the batch
    Notes:
        This loss function is not just a pure Negative Loss Likelyhood (NLL).
        ML algorythms require a way to tell the algorythm if it is learning or not. The most standard practice is with a Loss function.
        If the loss value goes down that means that the algorythm is learning and, if a step increases the loss, then it is punished and
        tries other approaches. When using uncertainties when teaching the model, the most common loss function is the NLL or Negative
        log-likelihood of a normal distribution 
        
        NLL$=\frac{1}{2}$log$(σ^2)+\frac{(y−μ)^2}{2σ^2}$
        
        where $\sigma$ is the uncertainty in the predictions, $y$ is the measured value and $\mu$ the predicted value.
        Instead of predicting $σ^2$ directly, we obtain its logarithm to have a more stable process (and avoid accounting precision as
        $\sigma^{-2}$ which is numerically unstable when uncertainties are low).

        However we want to avoid uncertainties that drift too far from the overall model predictions. To achieve that, we can get a rough
        estimate on what the uncertainty of a set predictions looks like. Let $\vec{\mu}=\left(\mu_1,\ldots,\mu_N\right)^T$ be the vector
        of $N$ predicted values. It can be considered as a random vector of variance:
        
        $Var(\vec{\mu})=\frac{1}{N}\sum_{i=1}^N\left(\mu_i-E(\vec{\mu})\right)$
        
        where $E(\vec{\mu})$ is the mean of the predicted values. We then have two different variances, one obtained as the sparseness of
        the predictions, (denoted as $Var\left(\vec{\mu}\right)$, and one obtained as a result of the internal ML calculations (denoted as
        $\sigma^2$). A penalty can be added to the loss functions to force the model to try to reduce this differences. An easy way to model
        it is to obtain the difference of those variances and square the result (taking the absolute value was also a good estimate, but
        using squared values punished big discrepancies in a stronger way).

        $StrayPenalty = B \cdot \left[\log\left(\sigma^2\right)-\log\left(\mathrm{Var}\left(\vec{\mu}\right)\right)\right]^2$
        
        where $B$ is a constant used to control the weight of this penalty. The reason why $Var\left(\vec{\mu}\right)$ was used and not
        $Var\left(\vec{y}\right)$ (with $\vec{y}$ the vector of measured values) was to avoid noise in the original data to tamper with the
        loss function. It would be physically clearer to use measured values sparseness as a way to guide the model but some experimental
        uncertainties are clearly underestimated and that would cause this penalty to dominate the loss and obscure the main loss protocol, the NLL.

        Also, we also want to punish the model if it tries overestimating $\sigma$. If the model is unable to minimize $y-\mu$, in order to
        lower NLL, it increases $\sigma$. If no precautions are taken, this "escape solution" achieves bad predictions with inflated
        uncertainties that simulate a low loss value. A new penalty was added that punishes overestimation of the uncertainties more than
        underestimation (which never happened). The slope correction done further on the pipeline can "fix" this issue but what the model
        returns then is closer to a poorly calculated linear fit. Therefore, an addition penalty was added.

        $OverestimatePenalty= C \cdot \max\left(0, \log\left(\sigma^2\right)-\log\left(\mathrm{Var}(\vec{\mu})\right)\right)$
        
        where $C$ is a constant used to control the weight of this penalty
    """

    mu = y_pred[:, 0:1]
    log_var = y_pred[:, 1:2]

    # Base Gaussian NLL
    precision = tf.exp(-log_var)
    nll = tf.reduce_mean(0.5 * (log_var + tf.square(y_true - mu) * precision))


    mu_centered = mu - tf.reduce_mean(mu)
    sigma_ref = tf.sqrt(tf.reduce_mean(tf.square(mu_centered)) + 1e-6)
    Stray_penalizer = 1e-2 #Parameter used to limit how far the uncertainty predictions stray from the experimental data
    OverUncert_penalizer  = 5e-3 #Parameter used for punishing overestimated uncertainties
    log_var_prior = tf.math.log(sigma_ref ** 2) #Order of magnitude of what uncertainties should look like

    # Prior penalty. It penalizes if uncertainty strays too much from the experimental value
    #It avoids underestimation of uncertainty and overestimation
    penalty_stray = Stray_penalizer * tf.reduce_mean(tf.square(log_var - log_var_prior))

    # Asymmetric penalty: punish overestimation more than underestimation
    penalty_overestimate = OverUncert_penalizer * tf.reduce_mean(tf.nn.relu(log_var - log_var_prior))

    return nll + penalty_stray + penalty_overestimate  

## 5. Model specific functions

Here we have functions that train, validate and fit the models. Some models require the variables to be scaled or will scale them. Extra precautions need to be taken into account

1. **model\_fitting**. It is a function that logs and runs model.fit() on a two-input Keras model and returns the training history. It needs **static, time and polarization variables scaled**. Training is not done using the uncertainties of the data as it was decided that uncertainty information is encoded in the augmentations. Note: No validation is done anywhere in the code. Here are some of the reasons:
    
    1.1. The data base is very small. The amorphous data base contains only 199 points while the crystalline one contains 251. Removing a small percentage of those points for validation might leave the data base too small and underfitting might worsen the result more than fine tuning parametrs with validation.
    
    1.2. A randomized validation split may be physically wrong. Therefore it should be chronological, not shuffled. However, in crystalline experiments, there are decay experiments that have only four or five intermediate points. Even removing one point for validation is a massive hit on the experiment. Therefore, it is risky to add validation
    
    1.3. To find good models, a Leave-one-out approach was used. For a certain model structure, an experiment gets removed and the model and it trains on all the remaining experiments. Then, the model tries to predict this isolated experiment. Afterwards, the experiment is returned and a new one becomes isolated. This process loops for all experiments and an overall score of the model is computed. This process was done for 498 models for crystalline materials. This is a stronger (and more expensive) method than validation as it is not dependent on the validation splits and avoids possible information leaks.

Also, eight randomly picked models were tested with and without validation and with and without an asymetric uncertainty-overestimated penalizing loss. The result showed that the Loss update was an improvement and validation did not increase performance (without validation, the results were slightly better).

2. **model\_prediction**. It is a funtion that predicts with a given model. It needs **static and time variables scaled**. This scaling must be coherent to the one done in the rest of the funtions.

3. **train**. This function is the one responsible of scaling the inputs and training the model (it uses **model\_fitting**)

     3.1. It creates the independent arrays with all the encoded experiments (augmented or not) using build_dataset
    
     3.2. Then it scales the data. ML algorithms work better when the inputs and outputs are normalized. The reason why we don't normalize inside the function is to have those scaler defined globally and not locally
    
     3.3. It builds the model depending on the use\_uncertainty bool. (It changes the loss function and the output).
    
     3.4. It trains the model and returns its history (the trained model)
    
4. **align_static_vectors**. It converts the columns not present on an isolated experiment to zeros.
    
5. **model_predict_sloped** It substracts a linear function to the predicted values. If done correctly this makes it so that the polarization predictions at the initial and final time points are the same as the measured polarization values at those times. This fixes a vertical shift and also an overall slope. As it is a correction done with experimental values, the algorithm is still "universal". However we can't fully say that it is a pure ML algorithm. The "correctness" of this method is subjective. It is a warning in the ML front that there is an issue with the data base but it is a valid fix for experimentalists.

In [ ]:
def model_fitting(model, X_static_scaled, X_time_scaled, y_scaled, epochs, batch_size, verbose, callbacks=None):
    """
    Arguments: 
        model (keras model): This is the object that the build_model function returns.
        X_static_scaled (array): It is an array os size (total number of intermediate measured points (let's call them 'samples'), number of static features). 
                                 Using the variables form build_dataset, the number of static features is 4+len(static_parameters(hot encoded)).
                                 To be precise, it is just Xs scaled (an array of lists with the static features) but reshpaed into a 2d array
        X_time_scaled (array): A 2d array of shape that reshapes Xt scaled from an array of lists (of size 1 like np.array([1],[13],[16],...)) to a 2D array
        y_scaled (array): The same as X_time_scaled but with polarization values
        epochs (int): Number of training epochs (times the model tries to validate the data and recalculates its parameters)
        batch_size (int): The number of samples for every gradient update
        verbose (int): It limits how much training output is printed
        callbacks (str): It allows certain Keras callbacks (special code properties of keras). EarlyStopping, ReduceLROnPlateau or ModelCheckpoint are examples
    Outputs:
       A keras.callbacks.History object
    Note:
        It logs and runs model.fit() on a two-input Keras model and returns the training history.
        IT REQUIRES THE INPUTS (static, time and polarization) TO BE NORMALIZED/SCALED
        Training is not done using the uncertainties of the data.  
    """
    log_message(f"    Model_fitting: Training the model.")
    return model.fit(
        [X_static_scaled, X_time_scaled],
        y_scaled,
        epochs=epochs,
        batch_size=batch_size,
        verbose=verbose,
        callbacks=callbacks)


def model_prediction(model, X_static_scaled, X_time_scaled):
    """
    Arguments:
        X_static_scaled (array): A 2d array of Xs (samples, number of static features)
        X_time_scaled (array): A 2d array of shape that reshapes Xt scaled from an array of lists (of size 1 like np.array([1],[13],[16],...)) to a 2D array
        y_scaled (array): The same as X_time_scaled but with polarization values. It doesn't have to be the same as the one used in training
    Output:
        A 2d array with the predictions for those time values. It contains a column of the polarization predictions and another with the log of the variance
    Notes:
        IT REQUIRES THE INPUTS (static and time) TO BE NORMALIZED/SCALED
    """
    log_message(f"    model_prediction: Predicting {len(X_time_scaled)} time points")
    return model.predict([X_static_scaled, X_time_scaled], verbose=0)

def train(encoded_experiments, scaler_static, scaler_time, scaler_y, use_uncertainty):
    """
    Arguments:
        encoded_experiments (list): A list like this:
            encoded_experiments = [
                (static_values,   # list of static parameters (per experiment)
                Deltatime,       # 1D numpy array of time values
                polarization,    # 1D numpy array of polarization values
                Uncertainty      # 1D numpy array of uncertainty values), ...] 
        scaler_static: It is a scikit-learn scaler for the static features. Only MinMaxScaler (normalizes uniformly using the maximum and the minimum)
        scaler_time: Analogously to scaler_static
        scaler_y: Analogously to scaler_static
        use_uncertainty (bool): It toggles whether it uses uncertainty for predictions or not. This changes the output of the model (mean and log_var or just the mean)
    Output:
        A fully trained model that can be used by model_predictions
    Notes:
        This function is the one responsible of scaling the inputs and training the model
        1. It creates the independent arrays with all the encoded experiments (augmented or not) using build_dataset
        2. Then it scales the data. ML algorithms work better when the inputs and outputs are normalized.
           The reason why we don't normalize inside the function is to have those scaler defined globally and not locally
        3. It builds the model depending on the use_uncertainty bool. (It changes the loss function and the output).
        4. It trains the model and returns its history (the trained model)
    
    """
    log_message(f"    train: Begin training and scaling with {len(encoded_experiments)} experiments.")
    X_static_all, X_time_all, y_all, u_all = build_dataset(encoded_experiments)
    log_message(f"Scaling all data")
    X_static_scaled = scaler_static.transform(X_static_all) #It only fits to the scaler. IT DOESN'T OVERWRITE THEM. WHAT A WASTE OF 20 DAYS OF MY LIFE >:(
    X_time_scaled = scaler_time.transform(X_time_all)
    y_scaled = scaler_y.transform(y_all)
    
    model = build_model(X_static_all.shape[1], use_uncertainty=use_uncertainty)
    epochs = 300
    batch_size = 32
    log_message(f"    train: Training final model with epochs={epochs}, batch_size={batch_size} and use_uncertainty = {use_uncertainty}")
    model_fitting(model, X_static_scaled, X_time_scaled, y_scaled, epochs, batch_size, verbose=0)
    return model

############################################################

def align_static_vectors(experiments, static_columns_training, static_columns_isolated):
    """
    Arguments:
        experiments (list): A list like this:
            experiments = [
                (static_values,   # list of static parameters (per experiment)
                Deltatime,       # 1D numpy array of time values
                polarization,    # 1D numpy array of polarization values
                Uncertainty      # 1D numpy array of uncertainty values), ...] 
        static_columns_training (list): A list of strings of all the static feature names used for training
        static_columns_isolated (list): A list of strings of all the static feature names of the isolated experiment
    Output:
        aligned_experiments is a list of experiments with static vectors aligned to the training feature order.
        To be precise (aligned_static, delta_time, polarization, uncertainty) where on the parameters where static_columns_isolated
        had no values get turned into zeros. So, if static_columns_isolated doesn´t have a parameter 'Parameter 6', then, in the
        original experiments lists, all numbers associated to 'Parameter 6' get turned to zero

    Notes:
        It helps the ML algorithm to focus on the important parameters
    """
    
    aligned_experiments = []
    for static, delta_time, polarization, uncertainty in experiments:
        static_dict = dict(zip(static_columns_isolated, static))
        aligned_static = [static_dict.get(col, 0.0) for col in static_columns_training]
        aligned_experiments.append((aligned_static, delta_time, polarization, uncertainty))
    log_message(f"    align_static_vectors: Vectors aligned")
    return aligned_experiments

def model_predict_sloped(model, X_static_scaled, X_time_scaled, m, n, scaler_y, scaler_time):
    """
    Arguments:
        model: A trained model
        X_static_scaled (array). A 2d array of Xs (samples, number of static features)
        X_time_scaled (array). A 2d array of shape that reshapes Xt scaled from an array of lists (of size 1 like np.array([1],[13],[16],...)) to a 2D array
        m (float): The slope used for correction
        n (float): The polarization shift for correction
        scaler_time: It is a scikit-learn scaler for the time arrays. Only MinMaxScaler (normalizes uniformly using the maximum and the minimum)
        scaler_y: Analogously to scaler_static
    Outputs
        y_pred_corrected_scaled is a 2D array where each column is the corrected predicted values (scaled) and the second one is the log of the variance (unchanged, a.k.a scaled)
    Notes:
        1. It predicts the polarization values
        2. It inverse transforms the predicted values and the time points (not the uncertainty)
        3. It calculates the correction curve in real units and corrects the predicted values.
        4. Finally, it scales back the corrected polarization values and joins them with the unchanged logarithm of the variance
        In summary, it substracts a linear function to the predicted values. If done correctly this makes it so that the polarization
        predictions at the time points stored as static features are the same as the measured polarization values at those times.
        This fixes a vertical shift and also an overall slope. As it is a correction done with experimental values, the algorithm is still
        universal. However we can't fully say that it is a pure ML algorithm.
    """
    
    log_message(f"    model_predict_sloped: Correcting {len(X_time_scaled)} points by subtracting P(t) = {float(np.squeeze(m)):.3e} * t + {float(np.squeeze(n)):.3e}")    # Step 1: Get scaled predictions from model
    y_pred_scaled = model_prediction(model, X_static_scaled, X_time_scaled)
    mean_scaled = y_pred_scaled[:, 0:1]  # shape (N,1)
    mean_real = scaler_y.inverse_transform(mean_scaled)  # shape (N,1)
    time_real = scaler_time.inverse_transform(X_time_scaled)  # shape (N,1)
    
    correction = m * time_real + n  # shape (N,1)
    mean_corrected_real = mean_real - correction  # shape (N,1)
    
    mean_corrected_scaled = scaler_y.transform(mean_corrected_real)  # shape (N,1)
    log_var_scaled = y_pred_scaled[:, 1:2]  # shape (N,1)
    y_pred_corrected_scaled = np.hstack([mean_corrected_scaled, log_var_scaled])  # shape (N,2)
    
    return y_pred_corrected_scaled

## 6 Predict function
The funtion that predicts using all stored models and all experiments.
    Outputs:
        A folder structure with the predictions:
        
        - PredictCrystalline.ipynb
        - CrystallinePredictionsFolder
          |
          |_Model_{NameOfModel 1}
          |    |
          |    |_Experiment_{NameOfExperiment 1}
          |    |   |
          |    |   |_Graph with predictions and original data (.png)
          |    |   |
          |    |   |_Text file with the results. First column is the internal program time
          |    |   |                             Second column is the time in real life of those predictions (YY-MM-DD HH:mm:SS
          |    |   |                             Third column is the predicted polarization
          |    |   |                             Fourth column is the uncertainty
          |    |   |
          |    |   |_PolarizationD3_{Name}_Paramteres.txt (original parameter file)
          |    |   |_PolarizationD3_{Name}.txt (original data file)
          |    |   
          |    |_Experiment_{NameOfExperiment 2}
          |    |   |
          |    |   |_Graph with predictions and original data (.png)
          |    |   |
          |    |   |_Text file with the results.  
          |    |   |
          |    |   |_Parameter and array original files
          |    |
          |    |_(...)
          |
          |_Model_{NameOfModel 2}
          |    |
          |    |_Experiment_{NameOfExperiment 1}
          |    |   |
          |    |   |_Graph with predictions and original data (.png)
          |    |   |_Text file with the results. First column is the internal program time
          |    |   
          |    |_Experiment_{NameOfExperiment 2}
          |    |   |
          |    |   |_Graph with predictions and original data (.png)
          |    |   |_Text file with the results.   
          |    |   |_Parameter and array original files
          |    |
          |    |_(...)
          |
          |_Combined model
          |    |
          |    |_Experiment_{NameOfExperiment 1}
          |    |   |
          |    |   |_Graph with predictions and original data (.png)
          |    |   |_Text file with the results. First column is the internal program time
          |    |   
          |    |_Experiment_{NameOfExperiment 2}
          |    |   |
          |    |   |_Graph with predictions and original data (.png)
          |    |   |
          |    |   |_Text file with the results.  
          |    |   |
          |    |   |_Parameter and array original files   
          |    |
          |    |_(...)
          |_(...)
    Notes:
    The steps it takes are explained here:
        1. Obtain the absolute time references for the experiments (year, month, day, hour, minute and second)
        2. Loop over all models that will be used
        3. Loop over all two-numerical-rowed experiments. It copies the original file to the final subfolders but doesn't move them
        4. Build the structure that the ML model was trained with and scale all the experiment using the scalers
           that the model used in training
        5. Predict, correct the slope if Correction=True, plot everything and write the results on a .txt file

In [ ]:
def Predict(predict_dir, model_dir, output_folder, time_dir, use_uncertainty, Correction, t_initial, t_final, t_step):
    """
    Arguments:
        predict_dir (str): String or path (Linux&Mac or Windows respectively) that points to the folder where the experiments that will be predicted are
        model_dir (str): String or path (Linux&Mac or Windows respectively) that points to the folder where the trained models are
        output_folder (str): String or path (Linux&Mac or Windows respectively) that points to the folder where the predicted experiments will be 
        time_dir (str): String or path (Linux&Mac or Windows respectively) that points to the file with the absolute time references
        use_unceratinty (bool): A bool that toggles whether uncertainty is used or not (not sure if it works on False but data without uncertainty is worthless)
        Correction (bool): A bool that toggles whether the initial and final point force gets done (works on True and False)
        t_initial (float): A float that controls the time point where predictions begin. For example, -1000 makes the predictions to start 1000 seconds before the first polarization measurement
        t_final (float): A float that controls the time point where predictions stop. For example, +1000 makes the predictions to stop 1000 seconds after the last polarization measurement
        t_step (float): A float that controls the time intervals between predictions (if it is 5, then there are predictions every 5 seconds)
    Outputs:
        A folder structure with the predictions:
        
        - PredictCrystalline.ipynb
        - CrystallinePredictionsFolder
          |
          |_Model_{NameOfModel 1}
          |    |
          |    |_Experiment_{NameOfExperiment 1}
          |    |   |
          |    |   |_Graph with predictions and original data (.png)
          |    |   |
          |    |   |_Text file with the results. First column is the internal program time
          |    |   |                             Second column is the time in real life of those predictions (YY-MM-DD HH:mm:SS
          |    |   |                             Third column is the predicted polarization
          |    |   |                             Fourth column is the uncertainty
          |    |   |
          |    |   |_PolarizationD3_{Name}_Paramteres.txt (original parameter file)
          |    |   |_PolarizationD3_{Name}.txt (original data file)
          |    |   
          |    |_Experiment_{NameOfExperiment 2}
          |    |   |
          |    |   |_Graph with predictions and original data (.png)
          |    |   |
          |    |   |_Text file with the results.  
          |    |   |
          |    |   |_Parameter and array original files
          |    |
          |    |_(...)
          |
          |_Model_{NameOfModel 2}
          |    |
          |    |_Experiment_{NameOfExperiment 1}
          |    |   |
          |    |   |_Graph with predictions and original data (.png)
          |    |   |_Text file with the results. First column is the internal program time
          |    |   
          |    |_Experiment_{NameOfExperiment 2}
          |    |   |
          |    |   |_Graph with predictions and original data (.png)
          |    |   |_Text file with the results.   
          |    |   |_Parameter and array original files
          |    |
          |    |_(...)
          |
          |_Combined model
          |    |
          |    |_Experiment_{NameOfExperiment 1}
          |    |   |
          |    |   |_Graph with predictions and original data (.png)
          |    |   |_Text file with the results. First column is the internal program time
          |    |   
          |    |_Experiment_{NameOfExperiment 2}
          |    |   |
          |    |   |_Graph with predictions and original data (.png)
          |    |   |
          |    |   |_Text file with the results.  
          |    |   |
          |    |   |_Parameter and array original files   
          |    |
          |    |_(...)
          |_(...)
    Notes:
    The steps it takes are explained here:
        1. Obtain the absolute time references for the experiments (year, month, day, hour, minute and second)
        2. Loop over all models that will be used
        3. Loop over all two-numerical-rowed experiments. It copies them (does't move them) to the final folders
        4. Build the structure that the ML model was trained with and scale all the experiment using the scalers
           that the model used in training
        5. Predict, correct the slope if Correction=True, plot everything and write the results on a .txt file
    """
    #Second run issues. Not important but needed
    def safe_copy(src: Path, dst: Path):
        src = src.resolve()
        dst = dst.resolve()
        if src == dst:
            return  # silently skip
        shutil.copy2(long_path(src), long_path(dst))

    model_path = Path(model_dir)
    predict_path = Path(predict_dir)
    output_folder_path = Path(output_folder)
    time_path = Path(time_dir)
    mode = "PolarizationD3"
    log_file = Path.cwd() / "CrystallineExecution_times.txt"
    log_file.parent.mkdir(parents=True, exist_ok=True)
    print(log_file)
    # 1. Read all polarization time references
    log_message(f"Predict: Obtain time references for all experiments")
    time_path_str = long_path(time_path) if os.name == "nt" else time_path 
    with open(time_path_str, "r", encoding="utf-8") as f:
        all_times = [datetime.strptime(line.strip(), "%Y-%m-%d %H:%M:%S") for line in f]

    # 2. Loop over all subdirectories inside model_dir (a.k.a obtain all models that will be used)
    for modelsubfolder in model_path.iterdir():
        log_message(f"Predict: Reading model {modelsubfolder}")
        if modelsubfolder.is_dir():
            try:
                folder_str = long_path(modelsubfolder)
                # 3. Find model files
                model_files = glob.glob(str(folder_str + "/model_*.keras"))
                scaler_static_files = glob.glob(str(folder_str + "/scaler_static_*.pkl"))
                scaler_time_files   = glob.glob(str(folder_str + "/scaler_time_*.pkl"))
                scaler_y_files      = glob.glob(str(folder_str + "/scaler_y_*.pkl"))
                model_file_path       = Path(model_files[0])
                scaler_static_path    = Path(scaler_static_files[0])
                scaler_time_path      = Path(scaler_time_files[0])
                scaler_y_path         = Path(scaler_y_files[0])
                
                model_file_path = Path(model_files[0])  # take the first matching model as a structure reference
                model = tf.keras.models.load_model(long_path(model_file_path), custom_objects={"nll_loss": nll_loss})
                scaler_static = joblib.load(long_path(scaler_static_path))
                scaler_time   = joblib.load(long_path(scaler_time_path))
                scaler_y      = joblib.load(long_path(scaler_y_path))
                log_message(f"\nPredict: Loaded model and scalers from {modelsubfolder}\n")

                #4. Get all *_Parameters.txt files to derive base names
                param_files = glob.glob(str(long_path(predict_path) + "/*_Parameters.txt"))
                param_files = [Path(f) for f in param_files] 
                base_names = [f.stem.replace("_Parameters", "") for f in param_files]
                output_midfolder_path = Path(output_folder) / modelsubfolder.name
                output_midfolder_path.mkdir(parents=True, exist_ok=True)  
                log_message(f"Predict: Created {output_midfolder_path}")
                output_midfolder_dir = long_path(output_midfolder_path)
                
                #5. Loop over all experiments 
                for i, base_name in enumerate(base_names):
                    start_time = pytime.time()    

                    file_param_path = Path(predict_dir) / f"{base_name}_Parameters.txt"
                    file_data_path  = Path(predict_dir) / f"{base_name}.txt"
                    PolarizationAbsoluteTime = all_times[i]  # pick the correct row of absolute times
                    log_message(f"Predict: [Experiment {i}] Polarization Reference = {PolarizationAbsoluteTime}")

                    parts = base_name.split("_")
                    short_name = "_".join(parts[1:6])  # take elements 1 through 5 only
                    output_subfolder_path = Path(output_midfolder_path) / f"Experiment_{short_name}"
                    Path(long_path(output_subfolder_path)).mkdir(parents=True, exist_ok=True)
                    if output_subfolder_path.exists():
                        for f in output_subfolder_path.glob("*"):
                            f.unlink()
                    else:
                        output_subfolder_path.mkdir(parents=True)
                    # 6 Move files to the experiment-specific folder. With this isolation, prediction begins
                    safe_copy(file_param_path, output_subfolder_path / file_param_path.name)
                    safe_copy(file_data_path,  output_subfolder_path / file_data_path.name)


                    log_message(f"Predict: Moved {base_name} files into {output_subfolder_path}")                
                    #7 Load isolated experiment. Predictions will be done on this experiment. Aligning is required
                    isolated_experiments, static_columns_isolated = load_experiments(output_subfolder_path, polarization_column=mode)
                    isolated_experiments_aligned = align_static_vectors(
                        isolated_experiments,
                        static_columns_training = ['Pressure', 'LabPolarization', 'LabTime', 'CellID_ge18004', 'CellID_ge18006', 'CellID_ge18012'],
                        static_columns_isolated=static_columns_isolated
                    ) #unscaled
                    if LogNoise:
                        log_message(f"      Number of isolated experiments aligned (should be one): {len(isolated_experiments_aligned)}")
                        log_message(f"      Example static vector (pre-scale): {isolated_experiments_aligned[0][0]}")
                        log_message(f"      Example time vector (pre-scale): {isolated_experiments_aligned[0][1]}")
                        log_message(f"      Example polarization vector (pre-scale): {isolated_experiments_aligned[0][2]}")



                    #8. Building manually the dataset for the isolated experiment
                    for static, timeVec, pol, unc in isolated_experiments_aligned: #all unscaled
                        log_message(f"{len(timeVec)}")
                        if len(timeVec) < 2:
                            log_message(f"**** Predict: Skipping experiment due to insufficient time points")
                            continue  # skip too-short experiments

                        #8.1 Extract the initial and final points, add them as parameters and fit the entire static vector
                        initial_dt, initial_p = timeVec[0], pol[0]
                        final_dt, final_p = timeVec[-1], pol[-1]
                        static_vector = static + [initial_dt, initial_p, final_dt, final_p] #unscaled
                        if LogNoise:
                            log_message(f"  Experiment static vector length (pre-scale): {len(static_vector)}")
                            log_message(f"  Experiment static vector (pre-scale): {static_vector}")
                        static_scaled = scaler_static.transform(np.array(static_vector).reshape(1, -1)).flatten() #scaled
                        if LogNoise:
                            log_message(f"  Experiment static vector (scaled): {static_scaled}")

                        #8.2 Scale time and polarization arrays (only intermediate points)
                        time_intermediate = np.array(timeVec).reshape(-1, 1) # unscaled
                        time_scaled = scaler_time.transform(time_intermediate).flatten() #scaled
                        if LogNoise:
                            log_message(f"  Experiment time (pre-scale): {time_intermediate.flatten()}")
                            log_message(f"  Experiment time (scaled): {time_scaled}")
                        pol_intermediate = np.array(pol).reshape(-1, 1) #unscaled
                        pol_scaled = scaler_y.transform(pol_intermediate).flatten() #scaled
                        if LogNoise:
                            log_message(f"  Experiment polarization (pre-scale): {pol_intermediate.flatten()}")
                            log_message(f"  Experiment polarization (scaled): {pol_scaled}")
                        unc_intermediate = np.array(unc).reshape(-1, 1) #unscaled
                        if LogNoise:
                            log_message(f"  Experiment uncertainty: {unc_intermediate.flatten()}")

                        #8.3 Combine all structures so that the shape is correct for prediction
                        scaled_isolated_experiments = []
                        scaled_isolated_experiments.append((
                            static_scaled, time_scaled, pol_scaled, unc_intermediate.flatten()
                        )) #scaled


                        #8.4 Check for shapes and create the final NumPy arrays of the experiment
                        X_static, X_time, y_true, u = [], [], [], []
                        for j, (static_scaled, time_scaled, pol_scaled, unc_intermediate) in enumerate(scaled_isolated_experiments):
                            if LogNoise:
                                log_message(f"   static_scaled.shape = {static_scaled.shape}")
                                log_message(f"   time_scaled.shape = {time_scaled.shape}")
                                log_message(f"   pol_scaled.shape = {pol_scaled.shape}")
                                log_message(f"   unc_intermediate.shape = {unc_intermediate.shape}")
                            assert len(time_scaled) == len(pol_scaled) == len(unc_intermediate), "Mismatch in lengths!"

                        for static_scaled, time_scaled, pol_scaled, unc_intermediate in scaled_isolated_experiments:
                            for t, p, err in zip(time_scaled, pol_scaled, unc_intermediate):
                                X_static.append(static_scaled)   # already includes appended time/polarization info
                                X_time.append([t])               # wrap to preserve 2D structure for Conv1D
                                y_true.append(p)
                                u.append(err)
                        X_static = np.array(X_static) #scaled
                        X_time = np.array(X_time) #scaled
                        y_true = np.array(y_true).reshape(-1, 1) #scaled
                        u = np.array(u).reshape(-1, 1) #unscaled
                        if LogNoise:
                            log_message(f"  X_static.shape: {X_static.shape}")
                            log_message(f"  X_time.shape: {X_time.shape}")
                            log_message(f"  y_true.shape: {y_true.shape}")
                            log_message(f"  X_static[0]: {X_static[0]} (scaled)")

                        #8.5 Extract the initial and final polarizations and keep a scaled and unscaled version of all. Then predict at those time points. 
                        #Unscaling needs to be done with static_scaler but then we need to scale them as polarizations or as time points with their respective scalers so that we can use model_predict
                        X_static_raw = scaler_static.inverse_transform(X_static)
                        # Extract static input (already scaled)
                        x_static_single = X_static[0:1]  # shape (1, D_static)
                        t0_raw = X_static_raw[0, -4].reshape(1, -1)
                        p0_raw = X_static_raw[0, -3].reshape(1, -1)
                        tT_raw = X_static_raw[0, -2].reshape(1, -1)
                        pT_raw = X_static_raw[0, -1].reshape(1, -1)

                        t0_scaled_correct = scaler_time.transform(t0_raw)
                        tT_scaled_correct = scaler_time.transform(tT_raw)
                        p0_scaled_correct = scaler_y.transform(p0_raw)
                        pT_scaled_correct = scaler_y.transform(pT_raw)

                        # Predict scaled polarizations at those time points
                        pred_initial_scaled = model_prediction(model, x_static_single, t0_scaled_correct)
                        pred_final_scaled   = model_prediction(model, x_static_single, tT_scaled_correct)

                        # Inverse transform predictions to physical (real) polarizations
                        p0_pred = scaler_y.inverse_transform(pred_initial_scaled[:, 0:1])
                        pT_pred = scaler_y.inverse_transform(pred_final_scaled[:, 0:1])

                        if LogNoise:
                            log_message(f"t0 (real) = {t0_raw}, tT (real) = {tT_raw}")
                            log_message(f"p0_pred (real) = {p0_pred}, pT_pred (real) = {pT_pred}")
                            log_message(f"p0 (true) = {p0_raw}, pT (true) = {pT_raw}")


                        #8.6 Prepare the linear correction and predict in the time points where we have data
                        m_raw = ( (p0_pred-p0_raw) - (pT_pred-pT_raw) ) / (t0_raw-tT_raw)
                        n_raw = (p0_pred-p0_raw) - m_raw * t0_raw


                        if Correction == True:
                            log_message(f"Predict:   Fixing initial and final points by substracting P(t) = {float(np.squeeze(m_raw)):.3e} * t + {float(np.squeeze(n_raw)):.3e}")
                            y_pred_raw = model_predict_sloped(model, X_static, X_time, m_raw, n_raw, scaler_y, scaler_time) #Input es scaled, parametros unsacles, output es scaled también
                        else:
                            y_pred_raw = model_prediction(model, X_static, X_time)

                        # 8.6 To obtain the results we separate the value predictions form the log variance. The results need to be unscaled to real units while the uncertainty needs to be 
                        #exponenciated (to obtain a variance), multiplied by the same factor that is used in MinMaxScaler and finally convert to uncertainty (take the square root)
                        mean_pred = scaler_y.inverse_transform(y_pred_raw[:, 0:1]).flatten()
                        log_var_scaled = y_pred_raw[:, 1]  # Keep in original scale
                        var_scaled = np.exp(log_var_scaled)
                        scale_y = scaler_y.data_max_[0] - scaler_y.data_min_[0]  # scale factor from MinMaxScaler 
                        # Variance rescales by the square of the scale factor
                        var_rescaled = var_scaled * (scale_y ** 2)
                        pred_sigma = np.sqrt(var_rescaled)
                        pred_polar = mean_pred

                        #Plots:
                        Grid_time_OG = np.arange(0+t_initial, float(tT_raw[0][0])+t_final, t_step).reshape(-1, 1)
                        static_scaled = np.tile(X_static[0], (len(Grid_time_OG), 1)) #SCALED
                        Grid_time_SC = scaler_time.transform(Grid_time_OG)

                        if Correction == True:
                            Grid_polar_Logvar_Sloped_SC = model_predict_sloped(model, static_scaled, Grid_time_SC, m_raw, n_raw, scaler_y, scaler_time) #scaled
                            Grid_polar_Sloped_OG = scaler_y.inverse_transform(Grid_polar_Logvar_Sloped_SC[:, 0:1]) #unscaled
                            Grid_Logvar_Sloped_SC = Grid_polar_Logvar_Sloped_SC[:, 1:2]  # Don't transform this
                            Grid_Var_Sloped_SC = np.exp(Grid_Logvar_Sloped_SC)
                            # Variance rescales by the square of the scale factor
                            Grid_Var_Sloped_OG = Grid_Var_Sloped_SC * (scale_y ** 2)
                            Grid_Uncert_Sloped_OG = np.sqrt(Grid_Var_Sloped_OG) #unscaled    
                        else:
                            Grid_polar_Logvar_SC = model_prediction(model, static_scaled, Grid_time_SC) #scaled
                            Grid_polar_OG = scaler_y.inverse_transform(Grid_polar_Logvar_SC[:, 0:1]) #unscaled
                            Grid_Logvar_SC = Grid_polar_Logvar_SC[:, 1:2]  # Don't transform this
                            Grid_Var_SC = np.exp(Grid_Logvar_SC)
                            # Variance rescales by the square of the scale factor
                            Grid_Var_OG = Grid_Var_SC * (scale_y ** 2)
                            Grid_Uncert_OG = np.sqrt(Grid_Var_OG) #unscaled   




                        fig, ax = plt.subplots(figsize=(8, 5)) 
                        title = f"{modelsubfolder.name}_{base_name}"
                        parts = title.split("_")
                        clean_title = "_".join([parts[1], parts[2]] + parts[4:9])  # keep a structure like "Average_3" + [234, polar1, 8, 12, 23, 0]
                        ax.set_title(clean_title)
                        ax.grid(True)

                        
                        if Correction == True:
                            ax.scatter(Grid_time_OG.flatten(), Grid_polar_Sloped_OG.flatten(), color='green', alpha=0.7, label="Predicted & Corrected", marker='x', s=10)
                            ax.fill_between(
                                Grid_time_OG.flatten(),
                                (Grid_polar_Sloped_OG - Grid_Uncert_Sloped_OG).flatten(),
                                (Grid_polar_Sloped_OG + Grid_Uncert_Sloped_OG).flatten(),
                                color='green',
                                alpha=0.1,
                                label="Prediction ±1σ")                            
                            
                             
                        else:
                            ax.scatter(Grid_time_OG.flatten(), Grid_polar_OG.flatten(), color='red', alpha=0.7, label="Predicted", marker='x', s=40)
                            ax.fill_between(
                                Grid_time_OG.flatten(),
                                (Grid_polar_OG - Grid_Uncert_OG).flatten(),
                                (Grid_polar_OG + Grid_Uncert_OG).flatten(),
                                color='red',
                                alpha=0.1,
                                label="Prediction ±1σ")

                        #Plot the initial and final points too
                        pred_times = np.array([initial_dt, final_dt])
                        pred_values = np.array([initial_p, final_p])
                        pred_uncert = np.array([u[0][0],u[1][0]]) 
                        ax.errorbar(
                            pred_times,
                            pred_values,
                            yerr=pred_uncert,
                            fmt='o',           # circle markers
                            color='black',
                            label='Original points ±1σ',
                            capsize=5,         # little caps on error bars
                            markersize=8
                        )
                        
                        # Tight Y-limits
                        #y_min = min(np.min(true_polar), np.min(mu_pred_grid)) - 0.05
                        y_min = float(pT_raw[0][0]) - 0.02
                        #y_max = max(np.max(true_polar), np.max(mu_pred_grid)) + 0.05
                        y_max = float(p0_raw[0][0]) + 0.02
                        ax.set_ylim([y_min, y_max])

                        ax.legend()
                        fig.tight_layout()

                        # Save figures
                        full_name = f"{modelsubfolder.name}_{base_name}"
                        parts = full_name.split("_")

                        # Keep "Average_3" (parts[1], parts[2]) and the chunk [234, polar1, 8, 12, 23, 0] (parts[4:10])
                        short_name = "_".join([parts[1], parts[2]] + parts[4:9])
                        fig_filename = short_name
                        fig_path = output_subfolder_path / fig_filename

                        predicted_data_path = output_subfolder_path / f"PredictedData_{short_name}.txt"

                        log_message(f"Predict: Saved figure to: {fig_path}")
                        fig.savefig(long_path(fig_path.with_suffix(".png")), dpi=200)
                        plt.close(fig)
                        del fig, ax
                        gc.collect()
                        log_message(f"Predict: Saved figure to: {fig_path.with_suffix('.jpg')}")


                        #9.Save data to text file

                        base_dir = os.path.dirname(os.getcwd())  # parent folder of ML
                        dst_path = long_path(os.path.join(os.getcwd(), "CopyPolarizationTimeReference.txt"))

                        
                        with open(long_path(predicted_data_path), 'w') as f:
                            f.write("GridTime,Time,PredictedPolarizationD3,ErrPredictedPolarizationD3\n")
                            if Correction == True:
                                for t, p, s in zip(Grid_time_OG.flatten(), Grid_polar_Sloped_OG.flatten(), Grid_Uncert_Sloped_OG.flatten()):
                                    # Compute absolute time as datetime + seconds
                                    abs_time = PolarizationAbsoluteTime + timedelta(seconds=float(t))
                                    f.write(f"{t:.6f},{abs_time.strftime('%Y-%m-%d %H:%M:%S')},{p:.6f},{s:.6f}\n")
                            else: 
                                for t, p, s in zip(Grid_time_OG.flatten(), Grid_polar_OG.flatten(), Grid_Uncert_OG.flatten()):
                                    # Compute absolute time as datetime + seconds
                                    abs_time = PolarizationAbsoluteTime + timedelta(seconds=float(t))
                                    f.write(f"{t:.6f},{abs_time.strftime('%Y-%m-%d %H:%M:%S')},{p:.6f},{s:.6f}\n")

                        log_message(f"Predict: Saved predicted data to: {predicted_data_path}")
                        end_time = pytime.time()
                        elapsed_time = end_time - start_time
                        with open(long_path(log_file), "a") as f:
                            f.write(f"Execution time={elapsed_time:.6f} seconds \n")

                del model, scaler_static, scaler_time, scaler_y

            except StopIteration:
                log_message(f"****Predict: Skipping {modelsubfolder} beacause the model files were not found.")                        
                        
                        

## 7. Main code and combined model

Runs the _Predict_ function. The code will not progress until the user manually inputs the time values. It asks for three values, t_initial (which means that predictions start _t_initial_ seconds later (it can be negative)), _t_final_ (it changes the final time point for predictions by t_final seconds) and _t_step_ (the time steps between predictions)
It also takes all previous predictions of all models and averages them using a weighted mean. That way we have a single prediction file that should be the best that ML can currently do.
For every experiment, the code finds all the experiment files created by each model. Then, for every line it performs the following calculation. Let $P_1\left(t\right),\ldots,P_n\left(t\right)$ be the polarization predictions of models $1,\ldots,n$ at time $t$ with uncertainties $\sigma_1\left(t\right),\ldots,\sigma_n\left(t\right)$. Then the weighted mean is 

$P\left(t\right)=\frac{\sum_{i=1}^n P_i\left(t\right)\sigma_i^{-2}\left(t\right)}{\sum_{i=1}^n \sigma_i^{-2}\left(t\right)}$

with

$\sigma\left(t\right)=\frac{1}{\sum_{i=1}^n \sigma_i^{-2}\left(t\right)}$

In [ ]:
# Define the directories relative to the script
model_dir = long_path(Path("./CrystallineModels/").resolve()) #Where models are stored
predict_dir = long_path(Path.cwd() / "CrystallineToPredictFiles") #Where experiments to predict are
output_folder_path = Path("CrystallinePredictionsFolder").resolve()
output_folder_path.mkdir(parents=True, exist_ok=True)
output_folder_dir = long_path(output_folder_path)

base_dir = Path.cwd().parent
time_dir = base_dir / "FileReadingStoring" / "PolarizationTimeReference.txt" #Where the absolute time of the experiments are
os.makedirs(model_dir, exist_ok=True)
os.makedirs(predict_dir, exist_ok=True)
os.makedirs(output_folder_dir, exist_ok=True)
if not os.listdir(model_dir):  
    raise RuntimeError(f"CrystallineModels is empty. Please add models to check again if the directory where models are is adequate")
if not os.listdir(predict_dir):  
    raise RuntimeError(f"CrystallineToPredictFiles is empty. Please add experiments to be predicted there like the ones in the example folder")
if not time_dir.exists():
    raise RuntimeError(f"No PolarizationTimeReference file found at {time_dir}. Run CrystallineFileLeacturePredict.ipynb again.")

Correction = True
use_uncertainty = True
def ask_number(prompt, cast=float): #Just to force the user to write the time values it wants to use for prediction
    while True:
        try:
            return cast(input(prompt))
        except ValueError:
            print("Invalid input, try again.")

t_initial = ask_number("Enter initial time (seconds): ", float)
t_final   = ask_number("Enter final time (seconds): ", float)
t_step    = ask_number("Enter time step (seconds): ", float)

Predict(predict_dir, model_dir, output_folder_dir, time_dir, use_uncertainty, Correction, t_initial, t_final, t_step)

# 1. Get all model prediction folders except the combined one
output_folder_path = Path(output_folder_path).resolve()  # absolute path
model_folders = [
    f for f in output_folder_path.iterdir()
    if f.is_dir() and f.name != "CombinedModel"
]
combined_dir = output_folder_path / "CombinedModel"

combined_dir = Path(combined_dir).resolve()
combined_dir.mkdir(parents=True, exist_ok=True)

if not model_folders:
    raise RuntimeError(f"No model prediction folders found in {output_folder_path}")

# 2. Read sub-subfolders using the first model folder as example
example_subs = model_folders[0] 
subsubfolders = [f for f in example_subs.iterdir() if f.is_dir()]

if not subsubfolders:
    raise RuntimeError(f"No experiment subfolders found in {example_subs}")

# 3. Load prediction files from all models
for subsub in subsubfolders:  # loop over each experiment folder
    data_frames = []
    
    # Load all .txt files for this subsubfolder across models
    for model_folder in model_folders:
        folder_path = model_folder / subsub.name
        txt_files = glob.glob(str(folder_path / "PredictedData_*.txt"))
        txt_files = [Path(long_path(f)) for f in txt_files]
        
        if len(txt_files) != 1:
            print(f"Skipping {folder_path}, expected 1 prediction file, found {len(txt_files)}")
            continue
        
        df = pd.read_csv(long_path(txt_files[0]))
        data_frames.append(df)

    # Verify all have same GridTime and Time columns
    grid_time = data_frames[0]["GridTime"]
    time_col = data_frames[0]["Time"]

    # Stack predictions and errors
    values = np.array([df["PredictedPolarizationD3"].values for df in data_frames])
    errors = np.array([df["ErrPredictedPolarizationD3"].values for df in data_frames])

    # Weighted mean
    weights = 1 / (errors ** 2)
    weighted_mean = np.sum(weights * values, axis=0) / np.sum(weights, axis=0)
    weighted_uncertainty = np.sqrt(1 / np.sum(weights, axis=0))

    # Create combined DataFrame
    combined_df = pd.DataFrame({
        "GridTime": grid_time,
        "Time": time_col,
        "PredictedPolarizationD3": weighted_mean,
        "ErrPredictedPolarizationD3": weighted_uncertainty
    })

    # Save in CombinedModel folder
    out_dir = combined_dir / subsub.name  
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / "PredictedPolarizationValues.txt"
    combined_df.to_csv(long_path(out_path), index=False)

    combined_df.to_csv(long_path(out_path), index=False)
    log_message(f"Saved combined file for {subsub.name}")

print("\nAll combined predictions created successfully!")
